In [ ]:
# Below we have sections of short driver scripts that run different parts of the heatmap code
# Not all functions in the 5 functiond definition .py's are called yet,
# but the ones that are called so far compile below without error

In [ ]:
import importlib
import setup_nx # your own module, setup.nx.py
import numpy as np
import math as m
import statistics as st
import cmath
import matplotlib.pyplot as plt 
import itertools
from operator import add
importlib.reload(setup_nx)
from setup_nx import *
from graphviz import Source, render

import datetime
import time

import my_feeder_funcs as ff
import my_impedance_funcs as imp
import my_configVis_funcs as vis
import my_detControlMatExistence_funcs as ctrl
import my_detLznRange_funcs as lzn
import my_heatmapSetup_funcs as hm


In [ ]:
# # List functions in each helper .py
# help(ff)
# print("----------------------------------------------------------")
# help(imp)
# print("----------------------------------------------------------")
# help(vis)
# print("----------------------------------------------------------")
# help(lzn)
# print("----------------------------------------------------------")
# help(hm)
# print("----------------------------------------------------------")
# help(ctrl)

In [ ]:
# [Essential] specify input feeder data

#-----------------------------------------------------------------
'specifying file paths'
# Enter the path/name of the model's excel file and import
    # All GridBright load files should be in the following folder
    #loadfolder = "/Users/jasperpakshong/Documents/Berkeley/ENERGISE/IEEE13/"
    #loadpath = loadfolder + "IEEE13testload_w_extreme_act.xlsx"

    # filepath = "IEEE13/"
    # modelpath = filepath + "001 phasor08_IEEE13_OPAL.xls"
    # loadfolder = "IEEE13/"
    # loadpath = loadfolder + "001_phasor08_IEEE13_norm03_HIL_7_1.xlsx"

    #filepath = "AL0001/"
    #modelpath = filepath + "AL0001_OPAL_working.xls"
    #loadfolder = "AL0001/"
    #loadpath = loadfolder + "AL0001_tvload_afternoon1h.csv"
      
    #filepath = "13NF_balanced/"
    #modelpath = filepath + "016 GB_IEEE13_balance_all_ver2.xls"
    #loadfolder = "13NF_balanced/"
    #loadpath = loadfolder + "016 GB_IEEE13_balance all ver2_time_sigBuilder_secondWise_norm03.csv"

filepath = "PL0001/"
modelpath = filepath + "PL0001_OPAL_working_reform_xfmr_heatmapMod.xlsx"
loadfolder = "PL0001/"
loadpath = loadfolder + "PL0001_July_secondwise_norm03.csv" # may update with better data soon

headerpath = '??13NF_balanced/001_phasor08_IEEE13_time_header.csv'
load_data = '??13NF_balanced/001_phasor08_IEEE13_time_PVpen50_cloud_minutewise (1).csv'


#==========================================================================================================

'specifying file name'
#file_name = string specifying name of dot file created when make_graph() is called
file_name = 'PL0001_test.dot'

#==========================================================================================================

'Specify substation kV, kVA bases, name, and the number of timesteps in the load data'
Vbase_ll = 7275*np.sqrt(3)
Vbase = 7275
Sbase = 1500000/3 #kVA of substation xfmr 
substation_name = 'bus_216866'
timesteps = 1

'DO NOT NEED TO EDIT THIS CELL BEFORE RUNNING'

ts = time.time()
print()
print(datetime.datetime.fromtimestamp(ts))

plot = 0 #turn plot on/off

depths = {}
leaves = []

In [ ]:
# [ESSENTIAL] create feeder obj
fin_feeder =ff.feeder_init(modelpath,loadfolder,loadpath,timesteps,Vbase_ll,Sbase,depths,leaves)
print("Finished initializing feeder")
ff.make_graph(fin_feeder, file_name)
node_index_map = hm.createNodeIndexMap(fin_feeder) #node indices for indicMat and F matrix
print(depths) # should be populated

R,X=hm.createRXmatrices_3ph(fin_feeder, node_index_map,depths)


count = 0 # print list of buses in network
for i in fin_feeder.network:    
    print(i) 
    count += 1
    if count >= 10:
        break
    
Source.from_file(file_name)
#^ need this to plot feeder

In [ ]:
#hm.get_path_to_substation(fin_feeder,'bus_N_L_21316_sec',depths)
print(len(depths))
print('depths=',depths)
# only secondary side buses connected to xfmrs show up in 'depths' and on network graph

In [ ]:
import csv 
graph = fin_feeder.network
#print(graph.nodes)

# write busnames into a csv
# with open("123NF_busList.csv", 'w', newline='') as csvfile:
#     spamwriter = csv.writer(csvfile, delimiter='-',
#                             quotechar='|', quoting=csv.QUOTE_MINIMAL)
#     spamwriter.writerows(graph.nodes)

In [ ]:
# [Optional] run impedance-related functions
slack_bus = None
for bus_name, depth in depths.items():
    if depth == 0:
        slack_bus = bus_name
        break
        
# -------------------- now we call functions: ---------------------------------------------
#print(depths)
# modify node names when change feeders
#plot_histogram_RX_ratios(fin_feeder, leaves_only = True)
print('Z between buses:')
print(np.around(imp.get_total_impedance_between_two_buses(fin_feeder, 'bus_033983', 'bus_053285',depths),2))
print('\nBus N_300053285 Z to substation:')
print(np.around(imp.get_total_impedance_from_substation(fin_feeder, 'bus_053285',depths),2))
print('\nR/X ratio of bus 49 to substation:')
print(imp.get_RX_ratio_tosubst(fin_feeder,'bus_053285',depths))
# should check on how to format the printing do that it's to like 2 decimal places

print(len(R))

In [ ]:
# R is of size 1023, which is 341*3, so set n=341
n=341
A, B = hm.setupStateSpace(n,fin_feeder, node_index_map,depths)
print('R = '+str(R))
print('X = '+str(X))
print('A = '+str(A))
print('B = '+str(B))
print(A.shape)
print(B.shape)

# write A and B matrices to csv
#np.savetxt("123NF_Amat.csv", A, delimiter=",")
#np.savetxt("123NF_Bmat.csv", B, delimiter=",")

# Save R and X matrices to csv to import into matlab
# np.savetxt reference: https://thispointer.com/how-to-save-numpy-array-to-a-csv-file-using-numpy-savetxt-in-python/
#np.savetxt('Rmat_PL0001.csv', R, delimiter=',')
#np.savetxt('Xmat_PL0001.csv', X, delimiter=',')

In [ ]:
import csv 
graph = fin_feeder.network
#print(graph.nodes)

#write busnames into a csv
with open("PL0001_busList.csv", 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter='-',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerows(graph.nodes)

In [ ]:
# ------- Create figure 2 ------------
phase_ratios = vis.phaseCouplingPerNode(fin_feeder,depths)
vis.createColorMap(fin_feeder, phase_ratios, 'fig2_PL0001')
Source.from_file('colorMap_fig2_PL0001') # display graph in notebook
ff.clear_graph(fin_feeder)

In [ ]:
# ---------- helper code to find index of any bus -----------
graph = fin_feeder.network
print(len(graph.nodes)) #number of nodes in network
print(len(depths)) # same as ^
print(substation_name)

bus2find=substation_name # you can choose any bus name
idx = [idx for idx, element in enumerate(graph.nodes) if element==bus2find]
print(idx)

In [ ]:
# commented out because takes LONG time to evaluate co-located at every node of 123NF
#-----------------------------------------------

# Create Fig 1: # start with just binary heat map across all nodes
# based on heat map colors, maybe try: branches color-coded and marked feasible (green outline) or infeasible (red outline) 

# find good colocated?
all_act_locs = []
perf_nodes = []
# empty act locs asks heatMapProcess to create heatmap for empty network, i.e. feas of placing 1 co-located actuator at each loc
feas_configs, heatMapNames=hm.find_good_colocated(fin_feeder, [], node_index_map,substation_name,depths, file_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)
print('----------- Feas configs are: ---------')
print(feas_configs)
Source.from_file(heatMapNames[0]) # display graph in notebook

In [ ]:
# ----- Run computeFParamSpace ---------
# all_act_locs = ['bus_25','bus_39','bus_56','bus_108'] # 49, then 76, then 65
# perf_nodes = ['bus_25','bus_39','bus_56','bus_108']
Fq_ub,Fp_ub=ctrl.computeFParamSpace_v2(fin_feeder, all_act_locs, perf_nodes,R,X,depths,node_index_map)
# this function prints Zgood (from R and X matrices) and Z_toSubst, where Z_toSubst is correct but doesnt match Zgood. Investigate
print('(Fp_ub,Fq_ub)=(',Fp_ub,',',Fq_ub,')')

In [ ]:
# run evalConfig on 10 different evenly spaced nodes
t = time.time()

all_act_locs = ['bus_053281'] 
perf_nodes = ['bus_053281']
feas, maxError,numfeas=hm.eval_config(fin_feeder, all_act_locs,perf_nodes, node_index_map,substation_name,depths,file_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)

all_act_locs = ['bus_062296'] 
perf_nodes = ['bus_062296']
feas, maxError,numfeas=hm.eval_config(fin_feeder, all_act_locs,perf_nodes, node_index_map,substation_name,depths,file_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)

all_act_locs = ['bus_004134'] 
perf_nodes = ['bus_004134']
feas, maxError,numfeas=hm.eval_config(fin_feeder, all_act_locs,perf_nodes, node_index_map,substation_name,depths,file_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)

all_act_locs = ['bus_020632'] 
perf_nodes = ['bus_020632']
feas, maxError,numfeas=hm.eval_config(fin_feeder, all_act_locs,perf_nodes, node_index_map,substation_name,depths,file_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)

all_act_locs = ['bus_062321'] 
perf_nodes = ['bus_062321']
feas, maxError,numfeas=hm.eval_config(fin_feeder, all_act_locs,perf_nodes, node_index_map,substation_name,depths,file_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)

all_act_locs = ['bus_033983'] 
perf_nodes = ['bus_033983']
feas, maxError,numfeas=hm.eval_config(fin_feeder, all_act_locs,perf_nodes, node_index_map,substation_name,depths,file_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)

all_act_locs = ['bus_062294'] 
perf_nodes = ['bus_062294']
feas, maxError,numfeas=hm.eval_config(fin_feeder, all_act_locs,perf_nodes, node_index_map,substation_name,depths,file_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)

all_act_locs = ['bus_233573'] 
perf_nodes = ['bus_233573']
feas, maxError,numfeas=hm.eval_config(fin_feeder, all_act_locs,perf_nodes, node_index_map,substation_name,depths,file_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)

all_act_locs = ['bus_062066'] 
perf_nodes = ['bus_062066']
feas, maxError,numfeas=hm.eval_config(fin_feeder, all_act_locs,perf_nodes, node_index_map,substation_name,depths,file_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)

all_act_locs = ['bus_063918'] 
perf_nodes = ['bus_063918']
feas, maxError,numfeas=hm.eval_config(fin_feeder, all_act_locs,perf_nodes, node_index_map,substation_name,depths,file_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)

elapsed = time.time() - t
print('Time elapsed=',elapsed)

In [ ]:
# run evalConfig on mult act config with perf node 10 different evenly spaced nodes
all_act_locs = ['bus_053281','bus_062296','bus_004134','bus_020632','bus_062321','bus_033983','bus_062294','bus_233573','bus_062066','bus_063918'] 
perf_nodes = ['bus_221986','bus_221986','bus_221986','bus_221986','bus_221986','bus_221986','bus_221986','bus_221986','bus_221986','bus_221986']
feas, maxError,numfeas=hm.eval_config(fin_feeder, all_act_locs,perf_nodes, node_index_map,substation_name,depths,file_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)


In [ ]:
# run evalConfig on mult perf colocated with 10 different evenly spaced nodes
all_act_locs = ['bus_053281','bus_062296','bus_004134','bus_020632','bus_062321','bus_033983','bus_062294','bus_233573','bus_062066','bus_063918'] 
perf_nodes = ['bus_053281','bus_062296','bus_004134','bus_020632','bus_062321','bus_033983','bus_062294','bus_233573','bus_062066','bus_063918'] 
feas, maxError,numfeas=hm.eval_config(fin_feeder, all_act_locs,perf_nodes, node_index_map,substation_name,depths,file_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)

In [ ]:
# ------ helper code to find indices of set of buses -----------
all_act_locs = ['bus_053281','bus_062296','bus_004134','bus_020632','bus_062321','bus_033983','bus_062294','bus_233573','bus_062066','bus_063918'] 
my_buses=list(node_index_map.keys())

config_indices=[]

for ele in all_act_locs:
    i = my_buses.index(ele)
    config_indices.append(i)

print(config_indices)


In [ ]:
# Compute RX ratios and phase coupling ratios here
#nodes = graph.nodes

# Print Table 
RXratios_10node=[]
for act in all_act_locs:
    dict=imp.get_RX_ratio_tosubst(fin_feeder,act,depths) # dictionary for the 3 phases
    RXratios_10node.append(sum(dict.values()) / len(dict)) # holds mean of RX ratio on each line

phVal=list(phase_ratios.values())
my_buses=list(node_index_map.keys())
phase_ratios_10node=[]
my_buses_10node=[]

for i in config_indices:
    phase_ratios_10node.append(phVal[i])
    my_buses_10node.append(my_buses[i])

#Table format: [bus_name, RX ratio, phase coupling ratio]
for my_buses_10node,RXratios_10node,phase_ratios_10node in zip(my_buses_10node,RXratios_10node,phase_ratios_10node):
    print(my_buses_10node,np.around(RXratios_10node,decimals=3),np.around(phase_ratios_10node, decimals=3))
    
# technically we should find the phase coupling ratio by splitting into R and X first...

In [ ]:
# -------Create Fig 4 ----------
# ----------- Demo of RHP on FEAS config --------------
 #complex config 1 (feas)
all_act_locs = ['bus_053281'] # 49, then 76, then 65
perf_nodes = ['bus_053281']
lst_feas_configs, lzn_error_run_sum,heatMapNames=hm.runHeatMapProcess(fin_feeder, all_act_locs, perf_nodes, node_index_map,substation_name,depths,file_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)
#see sections below to view graph results

#complex config 2 (feas)
# all_act_locs = ['bus_25','bus_39','bus_56','bus_108'] # 49, then 76, then 65
# perf_nodes = ['bus_25','bus_39','bus_56','bus_108']
# lst_feas_configs, lzn_error_run_sum,heatMapNames=hm.runHeatMapProcess(fin_feeder, all_act_locs, perf_nodes, node_index_map,substation_name,depths,file_name)
# time taken = 42561 seconds = 12 hours (though paused when went to sleep)

# t = time.time()
# #complex config 3 (feas)
# all_act_locs = ['bus_105','bus_112','bus_100','bus_91'] # 49, then 76, then 65
# perf_nodes = ['bus_105','bus_112','bus_100','bus_91']
# lst_feas_configs, lzn_error_run_sum,heatMapNames=hm.runHeatMapProcess(fin_feeder, all_act_locs, perf_nodes, node_index_map,substation_name,depths,file_name)
# #see sections below to view graph results
# # time taken = 42561 seconds = 12 hours (though paused when went to sleep)
# elapsed = time.time() - t
# print('Time elapsed=',elapsed)
      

In [ ]:
Source.from_file(heatMapNames[0]) # display graph in notebook

In [ ]:
Source.from_file(heatMapNames[1]) # display graph in notebook

Source.from_file(heatMapNames[2]) # display graph in notebook